### **Projet d'intelligence artificielle - Modèle de clustering et prédiction de résultat de match**

## **1 Description du projet**

**Contexte**

Au cours des vingt dernières années, les professionnels sportifs du monde entier ont adopté une approche fondée sur les données pour adapter leur prise de décision. Les analyses de métriques sportives sont aujourd’hui présentes partout, dans toutes les retransmissions en direct, dans l’eSport et même les discussions quotidiennes. Cette évolution a été notamment alimentée par une croissance exponentielle de la richesse de ces données.
Les équipes et les joueurs, de sport et d’eSport, ainsi que les parieurs professionnels font donc appel à la Data Science et au Machine Learning pour prendre les décisions les plus éclairées. Les sites de paris sportifs sont devenus très sophistiqués dans ce domaine, et certains modèles peuvent également être utilisés par les managers d'équipes ou des recruteurs pour constituer des équipes et déployer stratégiquement des joueurs de manière à maximiser leurs chances de victoire.
Le football a été au cœur de la révolution de l'analyse sportive. Tout type de métriques, à la fois historiques et en temps réel, sont aujourd’hui disponibles. Ce challenge s'appuie sur des données de football obtenues auprès de Sportmonks, un fournisseur majeur de données sportives largement utilisées dans ce domaine.
Ce projet est une modification de celui proposé par à challengedata.qube-rt.com

**But**

Dans le cadre du data challenge QRT de cette année, vous devez proposer
un modèle de clustering (vous pouvez choisir ce que vous voulez clusteriser : les joueurs, les matchs, les équipes).
un modèle de prédiction de résultat de match. Vous recevrez des données historiques réelles au niveau des équipes et des joueurs, et vous devrez prédire quelle équipe gagne ou s'il y a un match nul.

Les données couvrent de nombreuses ligues dans le monde entier ainsi que différentes divisions.